Processing CHIME FRB180814.J0422+73
=====

Windows restricts python's memory paging and causes a MemoryError 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import os
from tqdm import tqdm
import skimage.measure

folder = 'data/CHIME_FRB180814.J0422+73'
# burstdirs = ['burst180814', 'burst180911', 'burst180917', 'burst180919']
# burstdirs = ['burst180814', 'burst180911', 'burst180919']
burstdirs = ['burst180814']

res = {    #(tres, fres)
    'burst180814': (0.983, 16384/400),
    'burst180911': (0.983, 16384/400),
    'burst180917': (1.966, 16384/400),
    'burst180919': (0.983, 16384/400),
}

for burst in burstdirs:
    burstfile  = '{}_hstack.npy'.format(burst)
    weightfile = '{}weights_hstack.npy'.format(burst)

    cmap = plt.get_cmap('gray_r')
    cmap.set_bad(color = 'w', alpha = 1.)

    data    = np.load("{}/{}/{}".format(folder, burst, burstfile))
    weights = np.load("{}/{}/{}".format(folder, burst, weightfile))

    data    = data[:, :] 
    weights = weights[:, :]
    weights[13312:14592] = 0
    weights[11776:12032] = 0
    weights[2060:2850]   = 0
    weights[768:1024]    = 0

    np.putmask(data, ~weights.astype(bool), np.nan)
    data = data - 1*data[:, 0:200].mean(axis=1)[:,None]

    downsample = 256
    data = skimage.measure.block_reduce(data, block_size=(downsample, 1), func=np.nanmean)

    plt.figure(figsize=(10,10))
    plt.imshow(data[:,:], origin="lower", aspect="auto", interpolation="nearest", cmap=cmap, extent=(0, res[burst][0], 400, 800))
    plt.colorbar()
    plt.xlabel("Time [s]")
    plt.ylabel("Frequency [MHz]")

    plt.tight_layout()
    plt.show()
    #f burst == 'burst180814': plt.savefig('bursts180814.png')
    #if burst == 'burst180814': plt.savefig('bursts180814search.png')

MemoryError: Unable to allocate 256. MiB for an array with shape (64, 4096, 256, 1) and data type float32